# Работа с моделите от семейството на Meta

## Въведение

В този урок ще разгледаме:

- Двата основни модела от семейството на Meta – Llama 3.1 и Llama 3.2
- За какви случаи и сценарии е подходящ всеки модел
- Примерен код, който показва уникалните възможности на всеки модел

## Семейството модели на Meta

В този урок ще разгледаме два модела от семейството на Meta, наричано още "Llama Herd" – Llama 3.1 и Llama 3.2

Тези модели имат различни варианти и са достъпни на пазара за модели в Github. Повече информация за използването на Github Models за [прототипиране с AI модели](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) можете да намерите тук.

Варианти на моделите:
- Llama 3.1 – 70B Instruct
- Llama 3.1 – 405B Instruct
- Llama 3.2 – 11B Vision Instruct
- Llama 3.2 – 90B Vision Instruct

*Забележка: Llama 3 също е наличен в Github Models, но няма да бъде разглеждан в този урок*


## Llama 3.1

С 405 милиарда параметъра, Llama 3.1 попада в категорията на отворените LLM модели.

Този модел е надграждане на предишната версия Llama 3, като предлага:

- По-голям контекстен прозорец – 128k токена срещу 8k токена
- По-голям максимален брой изходни токени – 4096 срещу 2048
- По-добра многоезична поддръжка – благодарение на увеличения брой тренировъчни токени

Тези подобрения позволяват на Llama 3.1 да се справя с по-сложни случаи на употреба при създаване на GenAI приложения, включително:
- Вградени извиквания на функции – възможност за използване на външни инструменти и функции извън работния процес на LLM
- По-добра RAG производителност – благодарение на по-големия контекстен прозорец
- Генериране на синтетични данни – възможност за създаване на ефективни данни за задачи като фино настройване


### Извикване на собствени функции

Llama 3.1 е допълнително обучен, за да бъде по-ефективен при извикване на функции или инструменти. Моделът разполага и с два вградени инструмента, които може да разпознае като необходими според подадената от потребителя заявка. Тези инструменти са:

- **Brave Search** – Може да се използва за получаване на актуална информация, като например времето, чрез търсене в интернет
- **Wolfram Alpha** – Може да се използва за по-сложни математически изчисления, така че не е нужно да пишете собствени функции.

Можете също така да създадете свои собствени инструменти, които LLM да извиква.

В примерния код по-долу:

- Дефинираме наличните инструменти (brave_search, wolfram_alpha) в системния prompt.
- Изпращаме потребителска заявка, която пита за времето в определен град.
- LLM ще отговори с извикване на инструмент Brave Search, което ще изглежда така: `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Забележка: Този пример само извиква инструмента. Ако искате да получите резултатите, ще трябва да си създадете безплатен акаунт в страницата на Brave API и да дефинирате самата функция.*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Въпреки че е LLM, една от основните слабости на Llama 3.1 е липсата на мултимодалност. Тоест, невъзможността да използва различни типове вход, като например изображения, като подсказки и да предоставя отговори. Тази способност е една от основните характеристики на Llama 3.2. Тези възможности включват също:

- Мултимодалност – може да обработва както текстови, така и визуални подсказки
- Вариации с малък до среден размер (11B и 90B) – това осигурява гъвкави възможности за внедряване,
- Само текстови вариации (1B и 3B) – позволява моделът да се използва на edge / мобилни устройства и осигурява ниска латентност

Мултимодалната поддръжка е голяма крачка напред в света на отворения код. Примерът с код по-долу приема както изображение, така и текстова подсказка, за да получи анализ на изображението от Llama 3.2 90B.

### Мултимодална поддръжка с Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## Учението не спира тук, продължи своето пътешествие

След като завършиш този урок, разгледай нашата [колекция за обучение по Генеративен ИИ](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst), за да продължиш да развиваш знанията си в областта на Генеративния ИИ!



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, имайте предвид, че автоматичните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия изходен език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Не носим отговорност за недоразумения или погрешни тълкувания, произтичащи от използването на този превод.
